In [6]:
import random
import numpy as np
import pandas as pd
from copy import copy, deepcopy
from scipy.optimize import minimize
import itertools

import plotly
import plotly.graph_objs as go

import plotly.express as px
from plotly.subplots import make_subplots

from pathlib import Path
import json
import os

import sys
import time

In [25]:
# Возвращает функцию гаусса с заданными параметрами
def get_gaus_f(x,y,d):
    return lambda t: y*np.exp(-(t-x)**2/2/d**2)


# Возвращает спектр по параметрам гауссовых кривых
def EvalSol(sol, l_wavelength = np.linspace(0 ,1 , 1024), add_noise_disp = 0, mult_noise_disp = 0):
    X = l_wavelength
    spec = np.zeros(len(X))
    for n_gaus in range(0,len(sol), 3):
        f_gaus = get_gaus_f(sol[n_gaus], sol[n_gaus+1], sol[n_gaus+2])
        spec += np.array([f_gaus(t) for t in X])

    #Добавление шума
    add_spec_noise = np.random.normal(0, add_noise_disp, len(X))
    mult_spec_noise = np.random.normal(0, mult_noise_disp, len(X)) * spec
    spec = spec + add_spec_noise + mult_spec_noise

    return spec, X
    

# Рисует оба спектра и гауссианы
def DrawApprox(l_spec_1, l_spec_2, l_wavelength = np.linspace(0 ,1 , 1024), sol = np.array([])):
    fig = go.Figure()
    fig.update_layout(title_text="Spectrum",
                      title_font_size=20)
    
    K = l_wavelength
    fig.add_trace(go.Scatter(x=K,
                             y=l_spec_1,
                             name='1-st spec'))
    fig.add_trace(go.Scatter(x=K,
                             y=l_spec_2,
                             name='2-nd spec'))
    
    l_color = ['LimeGreen', 'MediumSlateBlue', 'Orange', 'DeepSkyBlue']
    i=0
    if len(sol) > 0:
        for n_gaus in range(0,len(sol), 3):
            f_gaus = get_gaus_f(sol[n_gaus], sol[n_gaus+1], sol[n_gaus+2])
            fig.add_vline(x=sol[n_gaus], line_width=2, line_color=l_color[i])
            fig.add_trace(go.Scatter(x=K,
                          y=[f_gaus(t) for t in K],
                          name=f'{n_gaus//3+1} gauss'))
            i+=1
    fig.show()
    return 


# Возвращает функцию Err(список-решение)
def GetFuncErr(spec, l_wavelength = np.linspace(0 ,1 , 1024)):
    F_err = lambda sol: np.sum((spec - EvalSol(sol)[0])**2)

    return F_err


def Grad_desc(f_Err, bounds, x0=None):
    if x0==None:
        x0=[]
        for a, b in bounds:
            x0.append(np.random.uniform(a,b))
        x0=np.array(x0)
        #print(f'x0: {x0}')

    res = minimize(f_Err, x0 = x0, bounds = bounds, method = 'SLSQP', options={'ftol':0})

    sol = res.x.tolist()
    f_err = f_Err(sol)

    return sol, f_err

In [8]:
sol_1 = [0.3, 0.6, 0.1, 
         0.6, 0.8, 0.1]

points_spec_1, X = EvalSol(sol_1)
F_err = GetFuncErr(points_spec_1, X)

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1]]
sol_2, fin_err = Grad_desc(f_Err=F_err, bounds=bounds, x0=None)
print(f'fin_err: {fin_err}\tsol: {sol_2}')
points_spec_2, X = EvalSol(sol_2)

DrawApprox(points_spec_1, points_spec_2, X, sol_2)

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_24024\1632630050.py:3: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_24024\1632630050.py:3: RuntimeWarning:

divide by zero encountered in scalar divide



fin_err: 2.4617524873213245e-12	sol: [0.5999999709860525, 0.8000000351421305, 0.10000001006634986, 0.29999997060717093, 0.6000000251004243, 0.09999996630059789]


# Эксперимент

In [22]:
def exper(init_sol, bounds, x0=None):
    '''Решение задачи, созданной гауссианами в init_sol.
    '''
    init_spec, X = EvalSol(init_sol)
    F_err = GetFuncErr(init_spec, X)

    sol, fin_err = Grad_desc(f_Err=F_err, bounds=bounds, x0=x0)

    return sol, fin_err


def mult_exper(disp_g, ampl_noise, bounds, num_launches=1):
    '''Проводит поиск решения c заданными дисперсиями гауссовых кривых и уровнем гауссового шумом.
    С помощью нескольких запусков.
    '''
    init_sol = [0.3, 0.6, disp_g, 
                0.6, 0.8, disp_g,
                
                0.5, ampl_noise, 0.3]
    
    print(f'\n=== disp_g: {disp_g}\tampl_noise: {ampl_noise} ===')

    best_sol, best_fin_err = exper(init_sol, bounds, x0=init_sol[:-3]) # 0-guess
    for i in range(num_launches):
        sol, fin_err = exper(init_sol, bounds)
        #print(f'{i}.\terr: {fin_err}\tx0: {sol}')
        if fin_err<best_fin_err:
            best_sol, best_fin_err = sol, fin_err
    print(f'= best_fin_err: {best_fin_err} =')
            

    return best_sol, best_fin_err

In [232]:
disp_g = 0.2
ampl_noise = 0.1

init_sol = [0.3, 0.6, disp_g, 
            0.6, 0.8, disp_g,
            0.5, ampl_noise, 0.3]

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1]]

sol, fin_err = exper(init_sol, bounds)


print(f'fin_err: {fin_err}\tsol: {sol}')

init_spec, X = EvalSol(init_sol)
fin_spec, X = EvalSol(sol)
DrawApprox(init_spec, fin_spec, X, sol)
DrawApprox(init_spec, fin_spec, X, init_sol)

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_26128\1632630050.py:3: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_26128\1632630050.py:3: RuntimeWarning:

divide by zero encountered in scalar divide



fin_err: 0.00020674497552503656	sol: [0.586929622557785, 0.9307596218484414, 0.20905156942360412, 0.28243246969864994, 0.5787489623901825, 0.19812129846905027]


In [208]:
np.linspace(0,0.3, 16)

array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
       0.22, 0.24, 0.26, 0.28, 0.3 ])

# 1-е приближение

In [215]:
'''l_disp_g = [0.1, 0.2, 0.3]
l_ampl_noise = [0.05, 0.1, 0.15, 0.2]
'''

l_disp_g = np.linspace(0, 0.3, 16)
l_ampl_noise = np.linspace(0, 0.5, 16)

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1]]


l_errors = []

for disp_g, ampl_noise in itertools.product(l_disp_g, l_ampl_noise):
    best_sol, best_fin_err = mult_exper(disp_g, ampl_noise, bounds = bounds, num_launches=5)
    l_errors.append(best_fin_err)

m_errors = np.array(l_errors).reshape(len(l_disp_g), len(l_ampl_noise))


=== disp_g: 0.0	ampl_noise: 0.0 ===


C:\Users\Gavriil\AppData\Local\Temp\ipykernel_26128\1632630050.py:3: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_26128\1632630050.py:3: RuntimeWarning:

invalid value encountered in scalar divide



= best_fin_err: 0.0 =

=== disp_g: 0.0	ampl_noise: 0.03333333333333333 ===
= best_fin_err: 3.7150220101735765e-14 =

=== disp_g: 0.0	ampl_noise: 0.06666666666666667 ===
= best_fin_err: 3.828881371883242e-14 =

=== disp_g: 0.0	ampl_noise: 0.1 ===
= best_fin_err: 2.1989063958524105e-14 =

=== disp_g: 0.0	ampl_noise: 0.13333333333333333 ===
= best_fin_err: 2.7304608651867833e-14 =

=== disp_g: 0.0	ampl_noise: 0.16666666666666666 ===
= best_fin_err: 1.3400263186980737e-14 =

=== disp_g: 0.0	ampl_noise: 0.2 ===
= best_fin_err: 3.3131456093239544e-14 =

=== disp_g: 0.0	ampl_noise: 0.23333333333333334 ===
= best_fin_err: 2.821408184713275e-14 =

=== disp_g: 0.0	ampl_noise: 0.26666666666666666 ===
= best_fin_err: 2.528936437968307e-14 =

=== disp_g: 0.0	ampl_noise: 0.3 ===
= best_fin_err: 3.042683243321602e-14 =

=== disp_g: 0.0	ampl_noise: 0.3333333333333333 ===
= best_fin_err: 4.696967869928698e-14 =

=== disp_g: 0.0	ampl_noise: 0.36666666666666664 ===
= best_fin_err: 4.562073032171408e-14 =

In [222]:
list(zip(list(itertools.product(l_disp_g, l_ampl_noise)), l_errors))

[((0.0, 0.0), 0.0),
 ((0.0, 0.03333333333333333), 3.7150220101735765e-14),
 ((0.0, 0.06666666666666667), 3.828881371883242e-14),
 ((0.0, 0.1), 2.1989063958524105e-14),
 ((0.0, 0.13333333333333333), 2.7304608651867833e-14),
 ((0.0, 0.16666666666666666), 1.3400263186980737e-14),
 ((0.0, 0.2), 3.3131456093239544e-14),
 ((0.0, 0.23333333333333334), 2.821408184713275e-14),
 ((0.0, 0.26666666666666666), 2.528936437968307e-14),
 ((0.0, 0.3), 3.042683243321602e-14),
 ((0.0, 0.3333333333333333), 4.696967869928698e-14),
 ((0.0, 0.36666666666666664), 4.562073032171408e-14),
 ((0.0, 0.4), 5.342566366077884e-14),
 ((0.0, 0.43333333333333335), 6.005600991543182e-14),
 ((0.0, 0.4666666666666667), 6.506415617313065e-14),
 ((0.0, 0.5), 1.0762943351480365e-12),
 ((0.02, 0.0), 36.26440578952688),
 ((0.02, 0.03333333333333333), 0.4032156670118032),
 ((0.02, 0.06666666666666667), 1.5878027401063464),
 ((0.02, 0.1), 3.508389012120886),
 ((0.02, 0.13333333333333333), 10.507408108055742),
 ((0.02, 0.166666666

In [218]:
fig = px.imshow(m_errors,
                labels=dict(y="disp_g", x="ampl_noise", color="Error"),
                y=l_disp_g,
                x=l_ampl_noise)

fig.show()

# 2-е приближение

In [233]:
'''l_disp_g = [0.1, 0.2, 0.3]
l_ampl_noise = [0.05, 0.1, 0.15, 0.2]
'''

l_disp_g = np.linspace(0, 0.15, 16)
l_ampl_noise = np.linspace(0.1, 0.7, 32)

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1]]


l_errors = []

for disp_g, ampl_noise in itertools.product(l_disp_g, l_ampl_noise):
    best_sol, best_fin_err = mult_exper(disp_g, ampl_noise, bounds = bounds, num_launches=5)
    l_errors.append(best_fin_err)

m_errors = np.array(l_errors).reshape(len(l_disp_g), len(l_ampl_noise))


=== disp_g: 0.0	ampl_noise: 0.1 ===


C:\Users\Gavriil\AppData\Local\Temp\ipykernel_26128\1632630050.py:3: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_26128\1632630050.py:3: RuntimeWarning:

invalid value encountered in scalar divide



= best_fin_err: 3.792440922678364e-14 =

=== disp_g: 0.0	ampl_noise: 0.11935483870967742 ===
= best_fin_err: 3.4994047862905354e-14 =

=== disp_g: 0.0	ampl_noise: 0.13870967741935486 ===
= best_fin_err: 3.650091494667269e-14 =

=== disp_g: 0.0	ampl_noise: 0.15806451612903227 ===
= best_fin_err: 3.525188320738493e-14 =

=== disp_g: 0.0	ampl_noise: 0.1774193548387097 ===
= best_fin_err: 2.7731047301485163e-14 =

=== disp_g: 0.0	ampl_noise: 0.1967741935483871 ===
= best_fin_err: 2.42691566844593e-14 =

=== disp_g: 0.0	ampl_noise: 0.2161290322580645 ===
= best_fin_err: 1.9208092517660466e-14 =

=== disp_g: 0.0	ampl_noise: 0.23548387096774195 ===
= best_fin_err: 3.795155814945667e-14 =

=== disp_g: 0.0	ampl_noise: 0.2548387096774194 ===
= best_fin_err: 3.88646249239765e-14 =

=== disp_g: 0.0	ampl_noise: 0.27419354838709675 ===
= best_fin_err: 3.6224933428402625e-14 =

=== disp_g: 0.0	ampl_noise: 0.2935483870967742 ===
= best_fin_err: 1.5425749663480296e-15 =

=== disp_g: 0.0	ampl_noise: 0.3

In [235]:
fig = px.imshow(m_errors,
                labels=dict(y="disp_g", x="ampl_noise", color="Error"),
                y=l_disp_g,
                x=l_ampl_noise)

fig.show()

In [15]:
disp_g = 0.06
ampl_noise = 0.2935484

init_sol = [0.3, 0.6, disp_g, 
            0.6, 0.8, disp_g,
            0.5, ampl_noise, 0.3]

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1]]

sol, fin_err = exper(init_sol, bounds)


print(f'fin_err: {fin_err}\tsol: {sol}')

init_spec, X = EvalSol(init_sol)
fin_spec, X = EvalSol(sol)
DrawApprox(init_spec, fin_spec, X, sol)
DrawApprox(init_spec, fin_spec, X, init_sol)

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_24024\1632630050.py:3: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_24024\1632630050.py:3: RuntimeWarning:

invalid value encountered in scalar divide



fin_err: 4.848253502231685	sol: [0.30137107891871057, 0.8008289466244721, 0.08106545180779355, 0.6008887025935068, 1.0, 0.0846696946214641]


# 3-е приближение

In [23]:
'''l_disp_g = [0.1, 0.2, 0.3]
l_ampl_noise = [0.05, 0.1, 0.15, 0.2]
'''

l_disp_g = np.linspace(0.001, 0.1, 32)
l_ampl_noise = np.linspace(0.1, 0.6, 64)

bounds = [[0,1], [0,1], [0,1], 
          [0,1], [0,1], [0,1]]


l_errors = []

for disp_g, ampl_noise in itertools.product(l_disp_g, l_ampl_noise):
    best_sol, best_fin_err = mult_exper(disp_g, ampl_noise, bounds = bounds, num_launches=5)
    l_errors.append(best_fin_err)

m_errors = np.array(l_errors).reshape(len(l_disp_g), len(l_ampl_noise))


=== disp_g: 0.001	ampl_noise: 0.1 ===


C:\Users\Gavriil\AppData\Local\Temp\ipykernel_24024\1632630050.py:3: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_24024\1632630050.py:3: RuntimeWarning:

divide by zero encountered in scalar divide



= best_fin_err: 1.7918197080717735 =

=== disp_g: 0.001	ampl_noise: 0.10793650793650794 ===
= best_fin_err: 1.7917543359114032 =

=== disp_g: 0.001	ampl_noise: 0.11587301587301588 ===
= best_fin_err: 1.1489453458384449 =

=== disp_g: 0.001	ampl_noise: 0.12380952380952381 ===
= best_fin_err: 1.1489584171932343 =

=== disp_g: 0.001	ampl_noise: 0.13174603174603175 ===
= best_fin_err: 1.1489650364013018 =

=== disp_g: 0.001	ampl_noise: 0.13968253968253969 ===
= best_fin_err: 0.6470773233180207 =

=== disp_g: 0.001	ampl_noise: 0.14761904761904762 ===
= best_fin_err: 1.7914820901880564 =

=== disp_g: 0.001	ampl_noise: 0.15555555555555556 ===
= best_fin_err: 1.791742006152428 =

=== disp_g: 0.001	ampl_noise: 0.1634920634920635 ===
= best_fin_err: 1.6093215467659454 =

=== disp_g: 0.001	ampl_noise: 0.17142857142857143 ===
= best_fin_err: 1.5043670483392009 =

=== disp_g: 0.001	ampl_noise: 0.17936507936507937 ===
= best_fin_err: 0.6470779720908926 =

=== disp_g: 0.001	ampl_noise: 0.187301587301

In [24]:
fig = px.imshow(m_errors,
                labels=dict(y="disp_g", x="ampl_noise", color="Error"),
                y=l_disp_g,
                x=l_ampl_noise)

fig.show()